In [24]:
from keras import backend as K
import tensorflow as tf
import numpy as np

In [26]:
from keras.losses import *

In [57]:
a = np.array([2.0, 5]) 
b = np.array([3.4, 5]) 
res = np.exp(a)
res

array([  7.3890561, 148.4131591])

In [99]:
from sklearn.metrics import r2_score
"""
1 - mean_squared_error(y_true,y_predict)/np.var(y_true)
"""
r2_score(a,b)

0.5644444444444445

In [101]:
from sklearn.metrics import mean_squared_error
mean_squared_error(a,b)

0.9799999999999999

In [96]:
def exp_loss(y_true,y_pred):
    return K.mean(K.exp(-y_true*y_pred))

In [8]:
def cos_distance(vects):
    x, y = vects
    fz = K.batch_dot(x,y,axes=1)
    fm1 = K.sum(K.square(x), axis=1, keepdims=True)
    fm2 = K.sum(K.square(y), axis=1, keepdims=True)
    fm = K.sqrt(fm1*fm2)
    return fz/K.maximum(fm, K.epsilon())

def triplet_loss(y_true, y_pred):
    """
    一次看三个样本
    Anchor, Positive, Negative
    L=max(d(a,p)−d(a,n)+margin,0)
    """
    MARGIN = 1.0  # 超参
    l = 768
# #     anc, pos, neg = y_pred[:, 0:128], y_pred[:, 128:256], y_pred[:, 256:]
    anc, pos, neg = y_pred[:, 0:l], y_pred[:, l:l*2], y_pred[:, l*2:]

#     # 欧式距离
    pos_dist = 1-K.abs(cos_distance((anc,pos)))
    neg_dist = 1-K.abs(cos_distance((anc,neg)))
#     pos_dist = K.sum(K.square(anc - pos), axis=-1, keepdims=True)
#     neg_dist = K.sum(K.square(anc - neg), axis=-1, keepdims=True)
#     pos_dist = y_pred[:, 0:1]
#     neg_dist = y_pred[:,1:]
    basic_loss = pos_dist - neg_dist + MARGIN
#     loss = basic_loss
    loss = K.maximum(basic_loss, 0.0)
#     print "[INFO] model - triplet_loss shape: %s" % str(loss.shape)
    return loss
    

In [49]:
def contrastive_loss(y_true, y_pred):
    """
    一次看两个样本
    对比损失
    y_pred: 两个样本欧式距离（0,1），距离越接近0，两个样本越相似
    y_true：y_true = 1 匹配，0： 不匹配
    Contrastive loss from http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [60]:
## emulation data
y_true = tf.constant([0.0, 0.0, 0.0, 0.0, 0.0])
y_pred = tf.constant([1.0, 1.0, 1.0, 1.0, 1.0])

In [48]:
K.eval(contrastive_loss(y_true,y_pred))

0.0

In [98]:
# binary_crossentropy
K.eval(binary_crossentropy(y_true, y_pred))

15.942385

In [97]:
K.eval(exp_loss(y_true,y_pred))

1.0

In [ ]:
import matplotlib.pyplot as plt
def show_loss(history):
    """
    loss 可视化
    """
#     print(history.history.keys())
    fig = plt.figure()
    plt.plot(history.history['acc'])
    plt.plot(history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['acc',"loss"], loc='upper left')